<a href="https://colab.research.google.com/github/B-veerababu/FMML_LABS-PROJECTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits of your dataset can provide more consistent and reliable results when assessing the performance of a machine learning model. This technique is commonly known as cross-validation, and it helps to address issues related to the random partitioning of data into training and validation sets.

Here's why averaging validation accuracy across multiple splits is beneficial:

1. **Reducing Variance**: By performing multiple splits (often called "folds") of the data and calculating validation accuracy separately for each fold, you can reduce the impact of random chance. If your dataset has inherent variability, using a single train-test split might lead to results that are not representative of the model's true performance. Cross-validation provides a more robust estimate of performance.

2. **More Comprehensive Evaluation**: Cross-validation allows you to evaluate the model on different subsets of the data. Each fold serves as a validation set, while the remaining folds are used for training. This provides a more comprehensive assessment of how well the model generalizes to unseen data.

3. **Detecting Overfitting**: Cross-validation can help you detect overfitting. If your model performs well on the training data but poorly on validation data across multiple folds, it may indicate that the model is overfitting the data.

4. **Parameter Tuning**: Cross-validation is often used in hyperparameter tuning (e.g., grid search or randomized search). It helps you select the best hyperparameters by assessing their impact on model performance across different data splits.

5. **More Confidence in Results**: Averaging results from multiple folds provides a more stable estimate of performance and gives you greater confidence in the reported accuracy or other evaluation metrics.

Common types of cross-validation include k-fold cross-validation (where the data is divided into k subsets or "folds"), stratified k-fold cross-validation (ensuring class distribution is maintained in each fold), and leave-one-out cross-validation (each data point serves as a test sample once).

In summary, averaging validation accuracy across multiple splits (cross-validation) is a best practice in machine learning because it provides more consistent and reliable performance estimates, which are crucial for making informed decisions about model selection and hyperparameter tuning.
2. Does it give more accurate estimate of test accuracy?

Cross-validation, while valuable for assessing the performance of a machine learning model and selecting appropriate hyperparameters, does not directly provide a more accurate estimate of the model's performance on unseen, real-world data. Instead, it provides a more reliable estimate of how the model might generalize to new data drawn from the same distribution as your training data. Here's why:

1. **Cross-Validation vs. Test Set**: Cross-validation estimates the model's performance on different subsets of your training data, but these subsets are still drawn from the same dataset that was used for training. It doesn't evaluate the model on entirely new, unseen data, as the test set does.

2. **Test Set Represents Real-World Scenario**: The test set serves as a proxy for how well your model will perform in a real-world scenario when presented with data it has never encountered before. It helps assess the model's generalization to new and unknown examples.

3. **Data Variability**: Cross-validation aims to reduce the impact of random variability in a single train-test split, but it doesn't address issues related to data distribution shifts, which can occur when your model encounters different data sources or environments.

4. **Bias and Variance**: Cross-validation can help you estimate the bias and variance of your model, but it doesn't directly measure how well the model will perform in a new, independent context. It provides insights into how well your model fits your training data, but it may not reveal issues that arise with entirely new data distributions.

In practice, it's important to use both cross-validation and a separate test set to obtain a comprehensive understanding of your model's performance. Cross-validation helps you fine-tune your model and hyperparameters, while the test set provides a final evaluation of how well your model is expected to perform in real-world situations. This approach ensures that your model's performance estimates are both reliable and indicative of its generalization capabilities.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

The number of iterations or epochs in the training of a machine learning model can significantly impact the model's performance and the accuracy of its estimates. However, it's not necessarily the case that higher iterations always lead to a better estimate. The relationship between the number of iterations and model performance can vary depending on several factors:

1. **Underfitting and Overfitting**: The number of iterations is closely related to the risk of underfitting or overfitting. If you have too few iterations, your model may underfit the data and not capture complex patterns. On the other hand, if you have too many iterations, your model might overfit the training data, capturing noise and leading to poor generalization on unseen data.

2. **Convergence**: Many machine learning algorithms, especially those based on optimization techniques like gradient descent, converge to an optimal solution with a sufficient number of iterations. After a certain point, increasing the number of iterations may not significantly improve the model's performance, as it has already reached a stable solution.

3. **Computational Resources**: Training with a high number of iterations can be computationally expensive, especially for deep learning models. It's essential to balance the computational cost with the expected performance gains.

4. **Early Stopping**: To mitigate the risk of overfitting, practitioners often use techniques like early stopping, where the training is halted when the model's performance on a validation set starts to degrade. This approach prevents excessive training.

5. **Hyperparameter Tuning**: The optimal number of iterations can depend on other hyperparameters, such as learning rate, batch size, and network architecture. It's common to perform hyperparameter tuning to find the best combination of settings, including the number of iterations.

In summary, the effect of the number of iterations on the estimate depends on the specific machine learning algorithm, dataset, and problem at hand. It's not a guarantee that higher iterations will lead to a better estimate, as there's a trade-off between underfitting and overfitting. It's essential to experiment with different iteration settings, monitor validation performance, and potentially use techniques like early stopping and hyperparameter tuning to determine the optimal number of iterations for your specific problem.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Increasing the number of iterations (or epochs) in training can help to a certain extent when dealing with a very small training dataset, but it may not fully mitigate the limitations of a small dataset. Here are some considerations:

**Advantages of Increasing Iterations with a Small Training Dataset:**

1. **Overfitting Mitigation**: Small datasets are more prone to overfitting because the model may learn to memorize the limited examples rather than generalize. Increasing the number of iterations can provide more opportunities for the model to see and adapt to the data, potentially reducing overfitting.

2. **Improved Training**: With more iterations, the model may converge to a better solution within the constraints of the available data. This can lead to better performance on the training set.

**Limitations and Considerations:**

1. **Limited Diversity**: Small datasets inherently have limited diversity and may not fully represent the complexity and variability of the real-world problem. Increasing iterations cannot create more diverse examples; it can only help the model better utilize the existing data.

2. **Risk of Memorization**: Even with increased iterations, there's a risk that the model may still memorize the training data, especially if the dataset is extremely small. Memorization can lead to poor generalization to new, unseen data.

3. **Validation Data**: If the validation dataset is also very small, increasing iterations might not be as effective because the model's ability to generalize to unseen data depends on the quality and diversity of the validation set.

4. **Data Augmentation**: In cases of small datasets, it's often more effective to apply data augmentation techniques, which artificially increase the effective size of the training dataset by creating variations of existing examples. Data augmentation can help improve model generalization even with a limited amount of original data.

In practice, when dealing with small datasets, it's essential to strike a balance between increasing iterations, applying regularization techniques (e.g., dropout, weight decay), using data augmentation, and monitoring the model's performance on a separate validation set. Combining these strategies can help improve the model's performance and mitigate the challenges posed by a small training dataset.